In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.decomposition import KernelPCA
def time_filereader(trainfile, rows, rate=0.3):
    x_f_v=[]
    x_f_t=[]
    y_f_v=[]
    y_f_t=[]
    t=False
    for _ in trainfile:
        
        file_root = r"C:\\Users\ksi03\Downloads\RP_data\real_validated\\"
        file=str(_)
        token=".csv"
        data_raw = pd.read_csv(file_root+file+token, sep=',', nrows=rows)
        
        data_raw=data_raw.dropna(axis=1)
        
        x_t=data_raw.iloc[0:int(data_raw.shape[0]*rate), 0:-2].values
        x_v=data_raw.iloc[int(data_raw.shape[0]*rate):, 0:-2].values
        
        y_t=data_raw.iloc[0:int(data_raw.shape[0]*rate), -2:].values
        y_v=data_raw.iloc[int(data_raw.shape[0]*rate):, -2:].values
        
        if t==False:
            x_f_v=x_v
            x_f_t=x_t
            y_f_v=y_v
            y_f_t=y_t
            t=True
        else:
            x_f_v=np.concatenate((x_f_v,x_v))
            x_f_t=np.concatenate((x_f_t,x_t))
            y_f_v=np.concatenate((y_f_v,y_v))
            y_f_t=np.concatenate((y_f_t,y_t))
        
    return x_f_v, x_f_t, y_f_v, y_f_t
        
    
def dropstring(df):
    df_tmp=df
    col_arr=df.columns[:]
    for col in col_arr:
        for row_n in range(df.shape[1]):
            if df[col].dtype=='O':
                df_tmp.drop(columns=col, axis=1)
    #print('processed dataframe is {}*{}'.format(df_tmp.shape[1], df_tmp.shape[0]))
    return df_tmp
        
def pointext(check_arr):
    loc=-1

    try:
        loc=list(check_arr[5:16]).index(0)
    except:
        print('index err')

    
    if loc==0:
        loc=101
    else: 
        loc=loc+5
        
    return loc

In [2]:
r=100
trainfile=[138, 696,1, 437]
#각각 10, 10, 15, 11

x_raw_valid, x_raw_train, y_raw_valid, y_raw_train=time_filereader(trainfile, rows=100, rate=0.3)


t_scaler=MinMaxScaler()
x_raw_train=t_scaler.fit_transform(x_raw_train)
x_raw_valid=t_scaler.transform(x_raw_valid)


print('x_raw_valid: {}'.format(x_raw_valid.shape))
print('y_raw_valid: {}'.format(y_raw_valid.shape))
print('x_raw_train: {}'.format(x_raw_train.shape))
print('y_raw_train: {}'.format(y_raw_train.shape))
#%matplotlib inline 
point=x_raw_train.shape[0]
#x_raw=np.concatenate((x_raw_train, x_raw_valid))
#print(x_raw.shape)

x_raw_valid: (280, 5122)
y_raw_valid: (280, 2)
x_raw_train: (120, 5122)
y_raw_train: (120, 2)


In [3]:

dim=5
pca1 = KernelPCA(n_components=dim, kernel='rbf')
x_raw_train = pca1.fit_transform(x_raw_train)
print(x_raw_train.shape)


(120, 5)


In [4]:
x_train=x_raw_train
x_valid=pca1.transform(x_raw_valid)

y_train=y_raw_train
y_valid=y_raw_valid


print('x_train_shape: {}'.format(x_train.shape))
print('x_valid_shape: {}'.format(x_valid.shape))

print('y_train_shape: {}'.format(y_train.shape))
print('y_valid_shape: {}'.format(y_valid.shape))

x_train=x_train.reshape(x_train.shape[0], 1, dim)
x_valid=x_valid.reshape(x_valid.shape[0], 1, dim)

print('x_train_reshape: {}'.format(x_train.shape))
print('x_valid_reshape: {}'.format(x_valid.shape))


x_train_shape: (120, 5)
x_valid_shape: (280, 5)
y_train_shape: (120, 2)
y_valid_shape: (280, 2)
x_train_reshape: (120, 1, 5)
x_valid_reshape: (280, 1, 5)


In [5]:
input_dim=dim
input_steps=1
hidden_dim=20
batch_size=1
best_model=0
#러닝
##########
##모델
init=tf.keras.initializers.he_uniform()
reg=tf.keras.regularizers.l2(l=0.001)
droprate = 0.2
#act=tf.nn.leaky_relu
act='relu'

model = tf.keras.Sequential(
[
#tf.keras.layers.Input(shape=(input_dim)),
tf.keras.layers.Input(shape=(input_steps, input_dim)),
tf.keras.layers.GRU(hidden_dim ,return_state=False, kernel_initializer=init, kernel_regularizer=reg),

#tf.keras.layers.Dense(50, activation=act),
#tf.keras.layers.Dropout(droprate),
tf.keras.layers.Dense(20, activation=act),
#tf.keras.layers.Dropout(droprate),
#tf.keras.layers.GaussianNoise(0.01),
tf.keras.layers.Dense(2, activation='softmax')])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            #optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss= 'categorical_crossentropy',
              metrics= ['accuracy'])

##
def variable_extracter(hist):
    loss = hist.history['loss'][0]
    acc=hist.history['accuracy'][0]

    return loss, acc
##러닝
current_cost_train = 1000 #초기 값으로 높은 값 부여
current_cost_valid = 1000 #초기 값으로 높은 값 부여
known_best = 1000000 #초기 값으로 높은 값 부여
full_step = 11
best_model=0
trainValues=[]
validValues=[]
for step in range(101): #3000번의 학습 후 종료

    
    train_hist = model.fit(x_train, y_train, batch_size=batch_size, verbose=1)
    loss, acc = variable_extracter(train_hist)
    
    if (step > 0 and step % 10 == 0): #매 50번째 학습 성능 값 출력
        print("[step: {}]'s cost = {}   acc={}".format(step, loss, acc)) 
        if (step >= 0): #1000번째부터 검증데이터로 모델 성능 평가
            current_cost_valid = model.evaluate(x_valid, y_valid, verbose=0)

            trainValues.append(loss) #학습 곡선을 출력하기 위해 현재 값 저장
            validValues.append(current_cost_valid[0]) #검증 곡선을 출력하기위해 현재 값 저장

            if (current_cost_valid[0] < known_best): #검증 성능이 이전보다 좋을 경우 현재 모델 저장
                known_best = current_cost_valid[0]
                best_model=model
                print("New best known cost = {}, acc={}".format(known_best, current_cost_valid[1]))
    
print("학습루틴 {}번 스텝 종료".format(step))

##
print("--- 학습 종료 ---")

model=best_model

Train on 120 samples
120/120 [==============================] - 2s 16ms/sample - loss: 0.7459 - accuracy: 0.6667
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.6200 - accuracy: 0.8000
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.4678 - accuracy: 0.9250
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.3262 - accuracy: 0.9583
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.2493 - accuracy: 0.9667
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.1941 - accuracy: 0.9583
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.1672 - accuracy: 0.9667
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.1503 - accuracy: 0.9667
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.1345 - accuracy

120/120 [==============================] - 0s 3ms/sample - loss: 0.0616 - accuracy: 0.9750
Train on 120 samples
120/120 [==============================] - 0s 3ms/sample - loss: 0.0450 - accuracy: 1.0000
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.0595 - accuracy: 0.9750
Train on 120 samples
120/120 [==============================] - 0s 3ms/sample - loss: 0.0518 - accuracy: 0.9917
[step: 70]'s cost = 0.051798833658297856   acc=0.9916666746139526
New best known cost = 0.022833454023514475, acc=1.0
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.0664 - accuracy: 0.9750
Train on 120 samples
120/120 [==============================] - 0s 3ms/sample - loss: 0.0837 - accuracy: 0.9583
Train on 120 samples
120/120 [==============================] - 0s 2ms/sample - loss: 0.0495 - accuracy: 0.9833
Train on 120 samples
120/120 [==============================] - 0s 3ms/sample - loss: 0.0586 - accuracy: 0.9833
Train o

In [6]:
print(validValues)

[0.04011300525494984, 0.033562432868140084, 0.029233992312635695, 0.02697263705943312, 0.025243713440639633, 0.024029811578137533, 0.022833454023514475, 0.02194012227867331, 0.021368877802576337, 0.02062700976218496]


In [7]:
#timing prediction start
r_final=[]
result=[]
resultlist=[]
filelist=[42, 209, 721, 814,272,465,602,692,15,162, 60]
flierst=[10,10,10,10,10,15,15,11,11,11, -1]
i=0
for _ in range(0, 828):
    try:
        file_root = r"C:\\Users\ksi03\Downloads\RP_data\train\\"
        file=str(_)
        token=".csv"
        test = pd.read_csv(file_root+file+token, sep=',', nrows=20)



        x_test=test.iloc[:, :].values
        x_test=t_scaler.transform(x_test)
        x_test_t= pca1.transform(x_test)
        x_test_t=x_test_t.reshape(x_test_t.shape[0], 1, x_test_t.shape[1])

        result_r=model.predict(x_test_t)
        resultlist.append(result[0:20])

        for _ in range(result_r.shape[0]):
            if result_r[_, 0]>result_r[_, 1]: result.append(0)
            else: result.append(1)

        loc=pointext(result)
        r_final.append([file,loc])
    except: r_final.append([_, 10])

import numpy as np
print(r_final)
t_model=model

[['0', 10], ['1', 10], ['2', 10], ['3', 10], ['4', 10], ['5', 10], ['6', 10], ['7', 10], ['8', 10], ['9', 10], ['10', 10], ['11', 10], ['12', 10], ['13', 10], ['14', 10], ['15', 10], ['16', 10], ['17', 10], ['18', 10], ['19', 10], ['20', 10], ['21', 10], ['22', 10], ['23', 10], ['24', 10], ['25', 10], ['26', 10], ['27', 10], ['28', 10], ['29', 10], [30, 10], ['31', 10], ['32', 10], ['33', 10], ['34', 10], ['35', 10], ['36', 10], ['37', 10], ['38', 10], ['39', 10], ['40', 10], ['41', 10], ['42', 10], ['43', 10], ['44', 10], ['45', 10], ['46', 10], ['47', 10], ['48', 10], ['49', 10], ['50', 10], ['51', 10], ['52', 10], ['53', 10], ['54', 10], ['55', 10], ['56', 10], ['57', 10], ['58', 10], ['59', 10], ['60', 10], ['61', 10], ['62', 10], ['63', 10], ['64', 10], ['65', 10], ['66', 10], ['67', 10], ['68', 10], ['69', 10], ['70', 10], ['71', 10], ['72', 10], ['73', 10], ['74', 10], ['75', 10], ['76', 10], ['77', 10], ['78', 10], ['79', 10], ['80', 10], ['81', 10], ['82', 10], ['83', 10], ['8

In [8]:
print(resultlist[-1])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
y_raw=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train_label.csv", sep=',')
def scalepca_pipeline(x, scaler, pca):
    x=scaler.transform(x)
    x=pca.transform(x)
    return x

def batch_generater(filenum, scaler, pca, nrows=None, rate=0.9, root=r"C:\Users\ksi03\Downloads\RP_data\train\\",):
    y_f=[]
    if nrows==None:
        x_raw=pd.read_csv(root+str(filenum)+".csv", sep=',').values
        #x_f=scalepca_pipeline(x_raw, scaler, pca)
        x_f=scaler.transform(x_raw)
        
        y=y_raw.iloc[filenum, 1]
        loc=r_final[filenum][1]
        for _ in range(loc):
            y_f.append(198)

        for i in range(x_f.shape[0]-loc):
            y_f.append(y)
    else:
        x_raw=pd.read_csv(root+str(filenum)+".csv", sep=',', nrows=nrows).values
        #x_f=scalepca_pipeline(x_raw, scaler, pca)
        x_f=scaler.transform(x_raw)
        
        y=y_raw.iloc[filenum, 1]
        loc=r_final[filenum][1]
        for stat_a in range(loc):
            y_f.append(198)

        for stat_b in range(nrows-loc):
            y_f.append(y)

        
    y_f=np.asarray(y_f)
    return x_f, y_f


def scaler_pca_generater(ref_filenum=100, dim=1000):
    #trainset인 0~745 이내에서 샘플을 100개 추출해 scaler와 pca 제작
    filelist=random.sample(range(0, 745), ref_filenum)
    scaler=MinMaxScaler()
    kpca = KernelPCA(n_components=dim, kernel='rbf')
    x_sample=[]
    i=False
    for num in filelist:
        if num!=30:
            if i!=False:
                tmp=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train\\"+str(num)+".csv", sep=',')
                tmp=tmp.iloc[10:]
                x_sample=data_raw=pd.concat([x_sample, tmp], ignore_index=True)
            else:
                x_sample=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train\\"+str(num)+".csv", sep=',')
        
    scaler.fit(x_sample)
    #kpca.fit(x_sample)
    kpca=False
    
    del x_sample
    return filelist, scaler, kpca

In [10]:
_, scaler, kpca=scaler_pca_generater(ref_filenum=100)

In [11]:
#828개의 dataset, 745개는 train, 43개는 valid, 40개는 test
#valid, train=상주변수 
#train=가변변수
y_raw=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train_label.csv", sep=',')
for i in range(746, 788):
    if i==746: 
        x_t, y_t=batch_generater(i, scaler, kpca)
        x_t=x_t[10:]
        y_t=y_t[10:]
    else: 
        x_t2, y_t2=batch_generater(i, scaler, kpca)
        x_t2=x_t2[10:]
        y_t2=y_t2[10:]
        
        x_t=np.concatenate((x_t, x_t2))
        y_t=np.concatenate((y_t, y_t2))
        
y_valid=y_t
x_valid=x_t

for i in range(789, 828):
    if i==746: 
        x_t, y_t=batch_generater(i, scaler, kpca)
        x_t=x_t[10:]
        y_t=y_t[10:]
    else: 
        x_t2, y_t2=batch_generater(i, scaler, kpca)
        x_t2=x_t2[10:]
        y_t2=y_t2[10:]
        x_t=np.concatenate((x_t, x_t2))
        y_t=np.concatenate((y_t, y_t2))
        
y_test=y_t
x_test=x_t

In [12]:
print(y_test)

[116 116 116 ... 110 110 110]


In [13]:
x_valid=x_valid.reshape(x_valid.shape[0], 1, x_valid.shape[1])
x_test=x_test.reshape(x_test.shape[0], 1, x_test.shape[1])

print(x_valid.shape)
print(y_valid.shape)
print(x_test.shape)
print(y_test.shape)

(24780, 1, 5122)
(24780,)
(47790, 1, 5122)
(47790,)


In [14]:
input_steps=1
hidden_dim=20
input_dim=5122
L_model = tf.keras.Sequential(
[
tf.keras.layers.Input(shape=(input_steps, input_dim)),
tf.keras.layers.GRU(hidden_dim ,return_state=False, kernel_initializer=init, kernel_regularizer=reg),
#tf.keras.layers.Dense(300, activation=act),
tf.keras.layers.Dense(200, activation=act),
#tf.keras.layers.Dropout(droprate),
tf.keras.layers.Dense(50, activation=act),
#tf.keras.layers.Dropout(droprate),
#tf.keras.layers.GaussianNoise(0.01),
tf.keras.layers.Dense(198, activation='softmax')])


L_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            #optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss= 'sparse_categorical_crossentropy',
              metrics= ['accuracy'])

In [15]:

_=True
for file in range(746):
    if file!=30:
        x_t, y_t=batch_generater(file, scaler, kpca, nrows=60)
        x_t=x_t[10:]
        y_t=y_t[10:]

        if _==True:
            x_train, y_train=x_t, y_t
        else:
            x_train=np.concatenate((x_train, x_t))
            y_train=np.concatenate((y_train, y_t))


        print('file{} is complete'.format(file))
        _=False
    
    
x_train=x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
print(x_train.shape)
print(y_train.shape)

file0 is complete
file1 is complete
file2 is complete
file3 is complete
file4 is complete
file5 is complete
file6 is complete
file7 is complete
file8 is complete
file9 is complete
file10 is complete
file11 is complete
file12 is complete
file13 is complete
file14 is complete
file15 is complete
file16 is complete
file17 is complete
file18 is complete
file19 is complete
file20 is complete
file21 is complete
file22 is complete
file23 is complete
file24 is complete
file25 is complete
file26 is complete
file27 is complete
file28 is complete
file29 is complete
file31 is complete
file32 is complete
file33 is complete
file34 is complete
file35 is complete
file36 is complete
file37 is complete
file38 is complete
file39 is complete
file40 is complete
file41 is complete
file42 is complete
file43 is complete
file44 is complete
file45 is complete
file46 is complete
file47 is complete
file48 is complete
file49 is complete
file50 is complete
file51 is complete
file52 is complete
file53 is complete
fil

file417 is complete
file418 is complete
file419 is complete
file420 is complete
file421 is complete
file422 is complete
file423 is complete
file424 is complete
file425 is complete
file426 is complete
file427 is complete
file428 is complete
file429 is complete
file430 is complete
file431 is complete
file432 is complete
file433 is complete
file434 is complete
file435 is complete
file436 is complete
file437 is complete
file438 is complete
file439 is complete
file440 is complete
file441 is complete
file442 is complete
file443 is complete
file444 is complete
file445 is complete
file446 is complete
file447 is complete
file448 is complete
file449 is complete
file450 is complete
file451 is complete
file452 is complete
file453 is complete
file454 is complete
file455 is complete
file456 is complete
file457 is complete
file458 is complete
file459 is complete
file460 is complete
file461 is complete
file462 is complete
file463 is complete
file464 is complete
file465 is complete
file466 is complete


In [16]:

##러닝
current_cost_train = 1000 #초기 값으로 높은 값 부여
current_cost_valid = 1000 #초기 값으로 높은 값 부여
known_best = 1000 #초기 값으로 높은 값 부여
full_step = 11
best_model=0
acc=0
trainValues=[]
validValues=[]
for file in range(50): #100회의 파일
    if file!=None:
        train_hist = L_model.fit(x_train, y_train, batch_size=300, verbose=1)
        loss, acc = variable_extracter(train_hist)

        if (file > 0 and file % 10 == 0): #매 10번째 학습 성능 값 출력
            print("[step: {}]'s cost = {}   acc={}".format(file, loss, acc)) 
            if (file >= 0): #1000번째부터 검증데이터로 모델 성능 평가
                current_cost_valid = L_model.evaluate(x_valid, y_valid, verbose=0)

                trainValues.append(loss) #학습 곡선을 출력하기 위해 현재 값 저장
                validValues.append(current_cost_valid[0]) #검증 곡선을 출력하기위해 현재 값 저장

                if (current_cost_valid[0] < known_best): #검증 성능이 이전보다 좋을 경우 현재 모델 저장
                    known_best = current_cost_valid[0]
                    best_model=L_model
                    print("New best known cost = {}, acc={}".format(known_best, current_cost_valid[1]))

        print("학습루틴 {}회 종료".format(file))


##
print("--- 학습 종료 ---")

label_model=best_model

Train on 37250 samples
37250/37250 [==============================] - 7s 187us/sample - loss: 3.4564 - accuracy: 0.3659
학습루틴 0회 종료
Train on 37250 samples
37250/37250 [==============================] - 5s 123us/sample - loss: 1.7346 - accuracy: 0.5932
학습루틴 1회 종료
Train on 37250 samples
37250/37250 [==============================] - 5s 126us/sample - loss: 1.4302 - accuracy: 0.6418
학습루틴 2회 종료
Train on 37250 samples
37250/37250 [==============================] - 4s 104us/sample - loss: 1.3500 - accuracy: 0.6543
학습루틴 3회 종료
Train on 37250 samples
37250/37250 [==============================] - 5s 146us/sample - loss: 1.2971 - accuracy: 0.6649
학습루틴 4회 종료
Train on 37250 samples
37250/37250 [==============================] - 5s 142us/sample - loss: 1.2858 - accuracy: 0.6668
학습루틴 5회 종료
Train on 37250 samples
37250/37250 [==============================] - 4s 104us/sample - loss: 1.2503 - accuracy: 0.6701 - loss: 1.248
학습루틴 6회 종료
Train on 37250 samples
37250/37250 [==============================] -

MemoryError: Unable to allocate 1.42 GiB for an array with shape (37250, 1, 5122) and data type float64

In [ ]:
#x_train.shape[0]
#이렇게 돌린 모델과 다른 모델 비교하기
tf.saved_model.save(L_model)

In [117]:
L_model=best_model
testcost=L_model.evaluate(x_test, y_test)
print(testcost[0])
_=True
for num in range(828, 929):
    x_raw=pd.read_csv(r'C:\Users\ksi03\Downloads\RP_data\test\\'+str(num)+".csv", sep=',').values
    x_real=scalepca_pipeline(x_raw, scaler, kpca)
    x_real=x_real.reshape(x_real.shape[0], 1, x_real.shape[1])
    if _==True: 
        y_real=L_model.predict(x_real, verbose=0) 
        _=False
    else:
        y_t=L_model.predict(x_real, verbose=0) 
        y_real=np.concatenate((y_real, y_t))
      


47790/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

6.023076935466999


ValueError: Shape of passed values is (6060, 198), indices imply (1, 198)

In [132]:
print(np.argmax(y_real[500]))

110


In [135]:
y_tmp=[]
for i in range(y_real.shape[1]):
    y_tmp.append(np.argmax(y_real[i]))


In [136]:
y_tmp

[0,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
